# 第3部 第3章 モデルを用いた予測

## はじめに

## フォルダ構造とユーティリティ関数、ライブラリimport
リンク集の記事にフォルダ構造とユーティリティ関数、ライブラリimportを掲載しました。\
準備としてそちらのページをご覧ください。
1. [フォルダ構造とユーティリティ関数]()
1. [ライブラリimport]()

## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
cmap = plt.get_cmap("tab10")
%matplotlib inline
import seaborn as sns
import arviz as az

# plotの設定
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}

file_path = "../mod/rcParams.json"
with open(file_path) as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 3.1 本章の目的と概要
参考書籍から引用します。\
RとStanの部分をPythonとNumPyroに読み替えてください。

> **テーマ**\
> 本章では単回帰モデルを用いた予測の方法を解説します。

> **目的**\
> モデルを用いた予測の方法の考え方を学ぶ目的で,本章を執筆しました。\
> 単回帰モデルを対象としますが,そのほかのモデルに対しても応用できるはずです。


> **概要**\
> 分析の準備 → 予測のためのデータの整理\
> → 予測のためのStanファイルの修正 → MCMCの実行 → 予測分布の可視化

## 3.2 分析の準備
前章の事後分布作成まで一気に行います。

In [ ]:
# データを読み込む
df = pl.read_csv("../data/3-2-1-beer-sales-2.csv")
#display(df.head())

# サンプルサイズを表示する
#print( len(df) )

In [ ]:
X = jnp.array(df["temperature"], dtype = float)
Y = jnp.array(df["sales"], dtype = float)

In [ ]:
def model_linear_regression(X, Y = None):
    '''
        第3部 2章 のビールの売り上げ
    '''
    # 説明変数をモデルに明示する
    # ベクトル化(学習用データを確率変数に割り当てるためのNumPyroのお作法)
    N = len(X)
    with numpyro.plate("N", N):
        X = numpyro.deterministic("X", X)
    # 1次関数の切片と傾きの事前分布を設定する
    β0 = numpyro.sample("β0", dist.Normal(loc = 0, scale = 20))
    β1 = numpyro.sample("β1", dist.Normal(loc = 0, scale = 20))
    # リンク関数(恒等関数)を定義する
    with numpyro.plate("N", N):
        μ = numpyro.deterministic("μ", β0 + β1 * X)
    # 目的変数が従う正規分布の標準偏差の事前分布を設定する
    σ = numpyro.sample("σ", dist.HalfNormal(scale = 20))
    # 目的変数 y は説明変数 x の値に応じた平均値 μ をパラメータとする正規分布に従うと仮定します。
    with numpyro.plate("N", N):
        numpyro.sample("Y", dist.Normal(loc = μ, scale = σ), obs = Y)

In [ ]:
model_args = {
    "X": X,
    "Y": Y
}
mcmc = run_mcmc(
    model_linear_regression,
    num_chains = 4,
    num_warmup = 1000,
    num_samples = 1000,
    thinning = 1,
    seed = 42,
    target_accept_prob = 0.9,
    **model_args
)
idata = az.from_numpyro(mcmc, log_likelihood = False)

## 3.3 単回帰モデルにおける予測の考え方
ビールの売り上げ $y_{i}$ と気温 $x_{i}$ との関係を再掲します。

$$
\begin{aligned}
y_{i} \sim& Normal(\mu_{i}, \sigma^2)\\
\mu_{i} \equiv& \beta_{0} + \beta_{1} x_{i}
\end{aligned}
$$

予測用の気温データ $x_{test}$ を与えたときのビールの売り上げ $y_{pred}$ は、同じ関係式を満たすと考えるのが自然です。

$$
\begin{aligned}
y_{pred} \sim& Normal(\mu_{pred}, \sigma^2)\\
\mu_{pred} =& \beta_{0} + \beta_{1} x_{test}
\end{aligned}
$$

サンプリングしたパラメータを上記のモデルに代入することで,予測値の分布を求めることができます。

## 3.4 予測のためのデータの整理
気温が11度から30度まで変化したときの売り上げの変化を調べます。

In [ ]:
X_test = jnp.array(range(11, 30 + 1))

## 3.5 予測のためのStanファイルの修正
NumPyroの場合,該当する内容が無いので省略する。

## 3.6 MCMCの実行

In [ ]:
# 予測用のデータ
model_args = {
    "X": X_test,
    "Y": None,
}
# 予測データの目的変数名
var_name = "Y"

# 事後予測分布を計算する
idata = compute_posterior_predictive_distribution(
    model_linear_regression,
    mcmc,
    var_name = "Y",
    seed = 42,
    hdi_prob = 0.95,
    log_likelihood = False,
    **model_args,
)

## 3.7 予測分布の可視化

In [ ]:
axes = az.plot_forest(
    idata["posterior_predictive"],
    var_names = "Y",
    combined = True,
    hdi_prob = 0.95
)
axes[0].set_title("図3.3.1  95%予測区間")

気温が11度の場合と30度の場合の比較は抽出方法が思い浮かばないのでスキップします。

## 終わりに